In [578]:
from datetime import datetime
from typing import Never, Generator
from unittest.mock import MagicMock

import freezegun
import ipytest
import pytest
import pytest_mock
import requests
from fastapi import FastAPI
from flask import Flask
from httpx import AsyncClient, Client
from mongomock_motor import AsyncMongoMockClient
from motor import MotorClient
from pytest_mock import MockerFixture, MockFixture


In [579]:
ipytest.autoconfig(
    rewrite_asserts=True, addopts=['-qq'],
    # run in a separate thread
    run_in_thread=True,
)


In [580]:
def turn_anything_to_list[T](x: T) -> list[T]:
    """
    Turn anything to list.
    Simple function that turns anything to list. 
    """
    if isinstance(x, list):
        return x
    else:
        return [x]

In [581]:
%%ipytest

def test_turn_anything_to_list():
    assert turn_anything_to_list(1) == [1]
    assert turn_anything_to_list([1]) == [1]
    assert turn_anything_to_list(2) != 2


.                                                                                            [100%]
========================================= warnings summary =========================================
../../Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204
  /Users/michael/Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: pytest_mock
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html


In [582]:
def turn_anything_to_list_but_not_a_list[T](x: T) -> list[T]:
    """
    Turn anything to list.
    Simple function that turns anything to list but will fail if given list. 
    """
    if isinstance(x, list):
        raise ValueError("I don't want a list")
    else:
        return [x]

In [583]:
%%ipytest

def test_turn_anything_to_list_but_not_a_list():
    assert turn_anything_to_list_but_not_a_list(1) == [1]
    # Here we test that the function raises an error using pytest.raises
    with pytest.raises(ValueError):
        turn_anything_to_list_but_not_a_list([1])

.                                                                                            [100%]
========================================= warnings summary =========================================
../../Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204
  /Users/michael/Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: pytest_mock
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html


In [584]:
"""
Example: unreachable service class test.
Here we have a classical case of why testing is hard we have some service can be in Azure can be an internal microservice that we want to call.
In CI and on our computer we can't or don't want to reach the service but we still want to test the code that calls the service.
"""


class SomeUnreachableService:
    def __init__(self, connection_string: str):
        self.connection_string = connection_string

    def client(self) -> Never:
        raise RuntimeError("Service not reachable")

    def make_call(self, arguments: list[str]) -> str:
        """
        Make a call to the service.
        Simple function that calls the service.
        """

        return self.client().call(arguments)[:-1]



In [585]:
%%ipytest

def test_call_unreachable_service():
    """
    We could go the route we went before to check if it raises an error.
    But the more interesting thing is to check was is the service called the way we expect.
    """
    with pytest.raises(RuntimeError):
        SomeUnreachableService(connection_string="something").make_call(["some", "arguments"])


# Here we will start using fixtures to mock the service's client to a Magic Mock
@pytest.fixture
def mock_service(mocker: MockerFixture):
    _some_unreachable_service_mocked = SomeUnreachableService(connection_string="connection_string")
    _some_unreachable_service_mocked.client = mocker.Mock()

    # we can also add a return value to the mock call
    _some_unreachable_service_mocked.client().call.return_value = "some return value"
    return _some_unreachable_service_mocked


def test_call_unreachable_service_with_mock(mock_service):
    # we can assert the call has the params we expect
    # Why is this important? Because we are processing the return value after we call the service.
    # The service is out of our control, but our code is ours, and we have to test it.
    assert mock_service.make_call(["some", "arguments"]) == "some return valu"

    # test that we called the function once
    mock_service.client().call.assert_called_once()

    # test that we called the function with the right arguments
    mock_service.client().call.assert_called_with(["some", "arguments"])


..                                                                                           [100%]
========================================= warnings summary =========================================
../../Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204
  /Users/michael/Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: pytest_mock
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html


In [586]:
"""
Example: unreachable service class test in real life.
Lets continue with previous example.
"""


def call_unreachable_service_dependency_injection(service: SomeUnreachableService, arguments: list[str]) -> None:
    """
    Call an unreachable service.
    Simple function that calls an unreachable service.
    """
    service.make_call(arguments)


In [587]:
%%ipytest

def test_call_unreachable_service_dependency_injection_with_mock(mock_service):
    """
    Now here, where the code uses nice dependency injection, we can test the code without too much hassle.
    We just provide our new mocked service to function, and we can test it.
    """
    call_unreachable_service_dependency_injection(mock_service, ["some", "arguments"])
    # test that we called the function once
    mock_service.client().call.assert_called_once()

    # test that we called the function with the right arguments
    mock_service.client().call.assert_called_with(["some", "arguments"])


.                                                                                            [100%]
========================================= warnings summary =========================================
../../Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204
  /Users/michael/Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: pytest_mock
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html


In [588]:
def call_unreachable_service_without_dependency_injection(arguments: list[str]) -> None:
    """
    Call an unreachable service.
    Simple function that calls an unreachable service but without dependency injection
    """
    service = SomeUnreachableService("connection_string")
    service.make_call(arguments)


In [589]:
%%ipytest

@pytest.fixture
def client_mock(mocker: MockerFixture) -> MagicMock:
    return mocker.MagicMock()


@pytest.fixture
def monkeypatch_service(monkeypatch: pytest.MonkeyPatch, client_mock: MagicMock) -> None:
    """
    Let's try to monkeypatch the service. 
    Here we also see an example of composing of pytest fixtures.
    We are taking the "client_mock" fixture as an argument and passing it to the "monkeypatch_service" fixture.
    """
    monkeypatch.setattr(SomeUnreachableService, "client", client_mock)


@pytest.mark.usefixtures("monkeypatch_service")
def test_call_unreachable_service_without_dependency_injection(client_mock: MagicMock):
    """
    "@pytest.mark.usefixtures("monkeypatch_service")" we start using more advanced features of pytest here.
    as we don't need the return value of the "monkeypatch_service" (which is None any way) and only need the "client_mock" fixture, we will pass one as an argument and one to the "pytest.mark.usefixtures" decorator.
    
    """
    call_unreachable_service_without_dependency_injection(["some", "arguments"])

    # test that we called the function once
    client_mock().call.assert_called_once()

    # test that we called the function with the right arguments
    client_mock().call.assert_called_with(["some", "arguments"])


.                                                                                            [100%]
========================================= warnings summary =========================================
../../Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204
  /Users/michael/Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: pytest_mock
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html


In [590]:
"""
Example 3 teardown in pytest.
In this example we have a Singleton class CountCalls class that counts its calls 
and we want to preform a few tests on.
"""


class Singleton(type):
    _instances = {}

    def __call__(self, *args, **kwargs):
        if self not in self._instances:
            self._instances[self] = super(Singleton, self).__call__(*args, **kwargs)
        return self._instances[self]


class CountCalls(metaclass=Singleton):
    call_count = 0

    def __init__(self):
        ...

    def make_call(self):
        self.call_count += 1


In [591]:
%%ipytest

def test_count_calls():
    """
    This test works perfectly
    """
    count_calls = CountCalls()
    count_calls.make_call()
    count_calls.make_call()
    assert count_calls.call_count == 2


@pytest.mark.xfail(reason="This test is expected to fail because the singleton is not cleaned between tests")
def test_failing_test():
    """
    While this one fails
    """
    count_calls = CountCalls()
    count_calls.make_call()
    count_calls.make_call()
    assert count_calls.call_count == 2


.x                                                                                           [100%]
========================================= warnings summary =========================================
../../Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204
  /Users/michael/Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: pytest_mock
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html


In [592]:
%%ipytest

# This will reset the call count for this test once
CountCalls().call_count = 0


@pytest.fixture
def call_count_with_teardown() -> Generator[CountCalls, None, None]:
    """
    This fixture will be called before and after each test that uses it.
    The yield statement is where the test will be run. After the yield statement, the teardown statement will be run.
    This is similar to how "@contextlib.contextmanager" works.
    * This might fail the first run as "CountCalls" was called and not cleaned before the test.
    """
    _count_calls = CountCalls()
    yield _count_calls
    _count_calls.call_count = 0


def test_count_calls_1(call_count_with_teardown):
    call_count_with_teardown.make_call()
    call_count_with_teardown.make_call()
    assert call_count_with_teardown.call_count == 2


def test_not_failing_test(call_count_with_teardown):
    call_count_with_teardown.make_call()
    call_count_with_teardown.make_call()
    assert call_count_with_teardown.call_count == 2


..                                                                                           [100%]
========================================= warnings summary =========================================
../../Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204
  /Users/michael/Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: pytest_mock
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html


In [593]:
"""
Example 4: mocker.ANY, freezetime and how to make tests more deterministic.
"""


def return_some_object_with_iso_datetime() -> dict[str, str]:
    return {"requested_at": datetime.now().isoformat(), "value": "some value"}


In [594]:
%%ipytest

@pytest.mark.xfail(reason="The 'requested_at' field is not the same as time moves")
def test_return_some_object_with_iso_datetime_failing():
    assert return_some_object_with_iso_datetime() == {
        "requested_at": datetime.now().isoformat(),
        "value": "some value",
    }


def test_return_some_object_with_iso_datetime_with_any(mocker: MockFixture) -> None:
    """
    Here we will use mocker special variable ANY.
    ANY will be equal to anything that is passed to it.
    """
    assert return_some_object_with_iso_datetime() == {
        "requested_at": mocker.ANY,
        "value": "some value",
    }


@freezegun.freeze_time("2021-01-01")
def test_return_some_object_with_iso_datetime_with_freezegun() -> None:
    """
    Here we will use freezegun.
    Freezegun will freeze the time to the date we pass to it.
    So we can make this test deterministic.
    """
    assert return_some_object_with_iso_datetime() == {
        "requested_at": "2021-01-01T00:00:00",
        "value": "some value",
    }

x..                                                                                          [100%]
========================================= warnings summary =========================================
../../Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204
  /Users/michael/Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: pytest_mock
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html


In [595]:
"""
Example 5: ASGI/WSGI application.
HTTPX back to the rescue with AsyncClient and Client we can provide any ASGI/WSGI application and call the app.
I created two super minimal apps one with FastAPI and one with Flask. but you can use any app you want as long as its ASGI/WSGI.
"""


def fastapi_app():
    app = FastAPI()

    @app.get("/hello")
    async def hello():
        return {"message": "Hello World"}

    return app


def flask_app() -> Flask:
    app = Flask(__name__)

    @app.route("/hello")
    def hello():
        return {"message": "Hello World"}

    return app



In [596]:
%%ipytest

@pytest.fixture(scope="session")
def fastapi_client_test() -> AsyncClient:
    return AsyncClient(base_url="http://test", app=fastapi_app())


@pytest.mark.asyncio
async def test_hello_route_fastapi(fastapi_client_test: AsyncClient):
    async with fastapi_client_test as client:
        response = await client.get("/hello")
        assert response.status_code == 200
        assert response.json() == {"message": "Hello World"}


@pytest.fixture(scope="session")
def flask_client_test() -> Generator[Client, None, None]:
    with Client(base_url="http://test", app=flask_app()) as client:
        yield client


def test_hello_route_flask(flask_client_test: Client):
    response = flask_client_test.get("/hello")
    assert response.status_code == 200
    assert response.json() == {"message": "Hello World"}

..                                                                                           [100%]
========================================= warnings summary =========================================
../../Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204
  /Users/michael/Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: pytest_mock
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html


In [597]:
"""
Example 6: Testing http calls and faking mongodb.
Before we talked about freezegun and how we can freeze time.
Lets see some more tools to help us test our code.
"""


def make_a_request_using_requests() -> requests.Response:
    """
    Make a request using requests.
    """
    return requests.get("http://localhost:3000/").json()


async def add_data_to_mongodb(motor_client: MotorClient) -> None:
    """
    Make an entire to mongodb.
    """
    db = motor_client.test
    collection = db.test_collection
    await collection.insert_one({"test": "test"})


In [598]:
%%ipytest

def test_make_a_request_using_requests(requests_mock):
    """
    Here we use the requests_mock fixture to mock the request. From the plugin "requests-mock".
    """
    requests_mock.get("http://localhost:3000/", json={"value": "hello mom"})
    assert make_a_request_using_requests() == {"value": "hello mom"}


@pytest.fixture
def async_motor_client():
    """
    Here we use the AsyncMongoMockClient to mock the motor client for mongodb. From the package "mongomock-motor".
    """
    return AsyncMongoMockClient()


@pytest.mark.asyncio
async def test_make_an_entire_to_mongodb(
    async_motor_client: AsyncMongoMockClient, mocker: pytest_mock.MockerFixture
):
    """
    Here we use the AsyncMongoMockClient to mock the motor client for mongodb. From the package "mongomock-motor". Plus, we use the mocker fixture to mock the _id of the data.
    """
    await add_data_to_mongodb(motor_client=async_motor_client)
    assert await async_motor_client.test.test_collection.find().to_list() == [
        {"_id": mocker.ANY, "test": "test"}
    ]


..                                                                                           [100%]
========================================= warnings summary =========================================
../../Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204
  /Users/michael/Library/Caches/pypoetry/virtualenvs/testing-the-untestable-0P_TvnzK-py3.12/lib/python3.12/site-packages/_pytest/config/__init__.py:1204: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: pytest_mock
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
